# Dependancies

In [1]:
import gpxpy
import gpxpy.gpx

import glob

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from geopy.distance import distance
from geopy.distance import geodesic
from geopy import distance
import math

import haversine as hs
import numpy as np

import datetime
from time import strftime
from time import gmtime

import folium
from folium.features import DivIcon

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
from IPython.display import display

# Enter one route

In [9]:
name = 'DD100K_2021'

with open('Routes/my_runs/'+name+'.gpx', 'r') as gpx_file:
    route = gpxpy.parse(gpx_file)
    
# making a dictionary that can be converted to Pandas Dataframe
route_info=[]

for track in route.tracks:
    for segment in track.segments:
        for point in segment.points:
            route_info.append({
                'time': point.time,
                'latitude': point.latitude,
                'longitude':point.longitude,
                'elevation': point.elevation
            })

df_single = pd.DataFrame(route_info)

In [6]:
df_1 = pd.read_csv('data/route_csv/Elizabeths_Furnace.csv')
df_2 = pd.read_csv('data/route_csv/vhtrc-waterfall-50k.csv')
df_ring = pd.read_csv('data/route_csv/vhtrc-ring.csv')
df_dd = pd.read_csv('data/my_route_csv/DD100K_2021.csv')
df_mtv = pd.read_csv('data/my_route_csv/2-loop_mtvernon.csv')




In [103]:
df_2.head()

,time,latitude,longitude,elevation,distance,elevation_change,cum_elevation,cum_distance,step_feet,mile_num,gradient_point,gradient_rating
0,NaN,38.64156,-78.58957,346.579987,0.000000,NaN,NaN,0.000000,0.000000,1.0,0.0,flat
1,NaN,38.64160,-78.58947,347.299683,0.006063,0.719696,0.719696,0.006063,32.013590,1.0,6.7,up
2,NaN,38.64158,-78.58935,346.940582,0.006622,-0.359101,0.360595,0.012685,34.963051,1.0,-3.1,flat
3,NaN,38.64159,-78.58921,347.120117,0.007587,0.179535,0.540130,0.020272,40.058568,1.0,1.4,flat
4,NaN,38.64163,-78.58909,347.839813,0.007041,0.719696,1.259826,0.027313,37.176850,1.0,5.8,up


In [10]:
print('gpx uphill ',route.get_uphill_downhill()[0])
print('WTF elevation up ', df_2[df_2['elevation_change'] >0]['elevation_change'].sum())
# print('WTF elevation down ', df_2[df_2['elevation_change'] <0]['elevation_change'].sum())
print('DD elevation up ', df_dd[df_dd['elevation_change'] >0]['elevation_change'].sum())
print('MTV elevation up ', df_mtv[df_mtv['elevation_change'] >0]['elevation_change'].sum())





gpx uphill  2837.000038357055
WTF elevation up  2823.9134350000027
DD elevation up  3020.400086402893
MTV elevation up  308.599990144372


# Miler Markers on route

In [ ]:
def find_neighbours(value, df, colname):
    exactmatch = df[df[colname] == value]
    if not exactmatch.empty:
        return exactmatch.index
    else:
        lowerneighbour_ind = df[df[colname] < value][colname].idxmax()
        upperneighbour_ind = df[df[colname] > value][colname].idxmin()
        return [lowerneighbour_ind, upperneighbour_ind] 

def make_mile_segments(row):
    return row['cum_distance']//1 +1
df['mile_num'] = df.apply(make_mile_segments, axis=1)

# find the begining location of each mile
def mile_marker_loc(df):
    mile_markers = []

    for i in range(1,len(df['mile_num'].unique())):
        junder,jover  = find_neighbours(i, df, 'cum_distance')
        mile_markers.append(jover)
    return mile_markers

def number_DivIcon(color,number):
    """ Create a 'numbered' icon
    
    """
    icon = DivIcon(
            icon_size=(150,36),
            icon_anchor=(14,40),
#             html='<div style="font-size: 18pt; align:center, color : black">' + '{:02d}'.format(num+1) + '</div>',
            html="""<span class="fa-stack " style="font-size: 12pt" >>
                    <!-- The icon that will wrap the number -->
                    <span class="fa fa-circle-o fa-stack-2x" style="color : {:s}"></span>
                    <!-- a strong element with the custom content, in this case a number -->
                    <strong class="fa-stack-1x">
                        {:02d}  
                    </strong>
                </span>""".format(color,number)
        )
    return icon
    
col_hex = ['#440154',
 '#481a6c',
 '#472f7d',
 '#414487',
 '#39568c',
 '#31688e',
 '#2a788e',
 '#23888e',
 '#1f988b',
 '#22a884',
 '#35b779',
 '#54c568',
 '#7ad151',
 '#a5db36',
 '#d2e21b']

# Graph mile markers on map with mile time on click - folium
lat_map = df['latitude'].mean()
lon_map = df['longitude'].mean()
route_map = folium.Map(
    location=[lat_map, lon_map],
    zoom_start=13,
    tiles='OpenStreetMap',
    width= 1000,
    height=800
)
coordinates = [tuple(x) for x in df[['latitude','longitude']].to_numpy()]
folium.PolyLine(coordinates, weight=6).add_to(route_map)

# find mile markers
locations_to_chart = mile_marker_loc(df)
# find mile times
# mile_times = run_miles(df)
num =0
mile=1

for i in locations_to_chart:
    mile_hex = mile//15+1
    loc = [df['latitude'][i], df['longitude'][i]]
    folium.Marker(
        location=loc,
        popup="Mile " + '{:02d}'.format(mile),
        icon=folium.Icon(color='white',icon_color='white'),
        markerColor=col_hex[mile_hex]
    ).add_to(route_map)

    # folium.Marker(
    #     location=loc,
    #     popup="Mile " + '{:02d}'.format(mile)+ " "+ mile_times[mile-1],
    #     icon= number_DivIcon(col_hex[mile_hex],mile)
    # ).add_to(route_map)
    # mile+=1

    folium.Marker(
        location=loc,
        popup="Mile " + '{:02d}'.format(mile),
        icon= number_DivIcon(col_hex[mile_hex],mile)
    ).add_to(route_map)
    mile+=1



display(route_map)

# Gradient Calculations

In [102]:
df_single.head()

,time,latitude,longitude,elevation
0,None,38.64156,-78.58957,346.579987
1,None,38.64160,-78.58947,347.299683
2,None,38.64158,-78.58935,346.940582
3,None,38.64159,-78.58921,347.120117
4,None,38.64163,-78.58909,347.839813


# Make DataFrame of all the routes

In [ ]:
# just on one df
how_run_grade2 =[]
grade_labels2 = []

diff_grades = df['gradient_rating'].unique()
for i in diff_grades:
    how_run_grade2.append(np.round(df[df['gradient_rating'] == i]['distance'].sum(),2))
    grade_labels2.append(i)
how_run_grade2.append(np.round(df['distance'].sum(),2))
grade_labels2.append('Total_Distance')
dict_list ={}
for i in how_run_grade2:
    dict_list[name] = how_run_grade2

test_df = pd.DataFrame.from_dict(dict_list,orient ='index',columns=grade_labels2)
test_df
# print(how_run_grade2)
# print(grade_labels2)

,flat,up,down,bigUp,bigDown,Total_Distance
vhtrc-waterfall-50k,7.78,1.21,1.29,0.13,0.14,10.53


In [94]:
# all the dfs
df_list = pd.read_csv('data/df_list.csv')
turn = 0
all_race_dict ={}
grade_labels_it = ['flat', 'up', 'down', 'bigUp', 'bigDown']
grade_labels3 = ['flat', 'up', 'down', 'bigUp', 'bigDown', 'Total_Distance']
all_elev_dict ={}
col_list = ['cum_elevation', 'cum_distance']


for r in df_list['races']:
    rdf = pd.read_csv('data/route_csv/'+r+'.csv')
    how_run_grade3 =[]
    # diff_grades = rdf['gradient_rating'].unique()
    for i in grade_labels_it:
        how_run_grade3.append(np.round(rdf[rdf['gradient_rating'] == i]['distance'].sum(),2))
    how_run_grade3.append(np.round(rdf['distance'].sum(),2))
    for i in how_run_grade3:
        all_race_dict[r] = how_run_grade3
    turn +=1



In [96]:
# all_race_dict
# print(grade_labels3)
all_race_df = pd.DataFrame.from_dict(all_race_dict,orient ='index',columns=grade_labels3)
all_race_df

,flat,up,down,bigUp,bigDown,Total_Distance
twot-2013,8.51,5.98,6.51,3.52,3.43,27.94
vickis-death-march,9.14,5.62,5.81,2.27,2.41,25.25
vhtrc-waterfall-50k,11.92,6.64,7.08,2.80,2.98,31.42
vhtrc-reverse-ring,27.89,15.83,15.65,5.26,5.44,70.06
vhtrc-mmt-training-two-2021,9.94,5.73,5.63,1.89,2.16,25.35
Elizabeths_Furnace,10.42,6.81,7.22,3.06,3.04,30.55
holy-cowans-gap,8.74,6.66,7.05,4.32,5.11,31.87
vhtrc-catherines-50km,11.19,7.19,7.11,2.11,2.35,29.95
vhtrc-martha-moats-baker,13.66,7.51,8.74,1.86,2.05,33.83
vhtrc-ring,27.89,15.83,15.65,5.26,5.44,70.06


In [97]:
all_race_df['up_ratio'] = np.round(((all_race_df['up']+all_race_df['bigUp'])/all_race_df['Total_Distance'])*100,2)
all_race_df['bigUp_ratio'] = np.round((all_race_df['bigUp']/all_race_df['Total_Distance'])*100,2)

all_race_df

,flat,up,down,bigUp,bigDown,Total_Distance,up_ratio,bigUp_ratio
twot-2013,8.51,5.98,6.51,3.52,3.43,27.94,34.00,12.60
vickis-death-march,9.14,5.62,5.81,2.27,2.41,25.25,31.25,8.99
vhtrc-waterfall-50k,11.92,6.64,7.08,2.80,2.98,31.42,30.04,8.91
vhtrc-reverse-ring,27.89,15.83,15.65,5.26,5.44,70.06,30.10,7.51
vhtrc-mmt-training-two-2021,9.94,5.73,5.63,1.89,2.16,25.35,30.06,7.46
Elizabeths_Furnace,10.42,6.81,7.22,3.06,3.04,30.55,32.31,10.02
holy-cowans-gap,8.74,6.66,7.05,4.32,5.11,31.87,34.45,13.56
vhtrc-catherines-50km,11.19,7.19,7.11,2.11,2.35,29.95,31.05,7.05
vhtrc-martha-moats-baker,13.66,7.51,8.74,1.86,2.05,33.83,27.70,5.50
vhtrc-ring,27.89,15.83,15.65,5.26,5.44,70.06,30.10,7.51
